In [1]:
!pip install pyquaternion

In [5]:
from bokeh.io import output_notebook, show, curdoc
curdoc().theme = 'light_minimal'
output_notebook()
from bokeh.plotting import figure
 
import numpy as np
import numpy.matlib
from numpy import random

from scipy.integrate import solve_ivp
from scipy.optimize import minimize, fminbound
from scipy.interpolate import pchip_interpolate

import copy

from pyquaternion import Quaternion

import math

# Attitude Determiniation and Control Systems

## Kinematics Equation

In [12]:
def kinematics(attitude = Quaternion(1), rate=np.array((0.0, 0.0, 0.0))):
  return attitude.derivative(rate)

In [13]:
attitude = Quaternion(1)
rate = np.radians(np.array((0.2, 0.0, 0.0)))

attitude_derivative = kinematics(attitude, rate)

In [16]:
# Quaternion kinematics are handled by the Quaternion class
print("\nattitude derivative: {0}".format(attitude_derivative))


attitude derivative: +0.000 +0.002i +0.000j +0.000k


## Dynamics Equation

In [10]:
def dynamics(inertia=np.eye(3), torques=np.array((0.0, 0.0, 0.0)), rate=np.array((0.0, 0.0, 0.0)), angular_momenta=np.array((0.0, 0.0, 0.0))):
  return np.linalg.inv(inertia) @ (torques - np.cross(rate, angular_momenta))

In [ ]:

inertia = 2000e-6 * np.eye(3)

torques = np.array((0.0, 0.0, 0.0))
angular_momenta = np.array((0.0, 0.1, 0.0))

rate_derivative = dynamics(inertia, torques, rate, angular_momenta)

In [ ]:

rate = np.radians(np.array((0.2, 0.0, 0.0)))

torques = np.array((0.0, 0.0, 0.0))
angular_momenta = np.array((0.0, 0.1, 0.0))

rate_derivative = dynamics(inertia, torques, rate, angular_momenta)

In [8]:
print("inertia:\n{0}".format(inertia))
print("\nattitude: {0}".format(attitude))
print("\nrate: {0}".format(np.degrees(rate)))

# dynamics are defined by above function
print("\nrate derivative: {0}".format(rate_derivative))

inertia:
[[0.002 0.    0.   ]
 [0.    0.002 0.   ]
 [0.    0.    0.002]]

attitude: +1.000 +0.000i +0.000j +0.000k

rate: [0.2 0.  0. ]

attitude derivative: +0.000 +0.002i +0.000j +0.000k

rate derivative: [ 0.          0.         -0.17453293]


## pFDA Model

In [6]:
def pfda(t, x, u, p):
  '''pFDA dynamics'''
  return np.array([x[1] + p[0]/p[1] * u, -1/p[1] * (x[1] + p[0]/p[1] * u)]).flatten()

pa = [80e-6, 0.5]   # pFDA parameters
ua = 1              # control

# solve the initial value problem
sol = solve_ivp(pfda, [0, 5], [0, 0], args=(ua, pa), first_step=0.1, max_step=0.1)

# grab simulation output and create output vector from simulated state vector
t = sol.t
xa = sol.y
ya = copy.deepcopy(xa)
ya[1,:] = ya[1,:] + pa[0]/pa[1]*ua